In [0]:
!pip install -q http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl torchvision
!pip install -q keras

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.cuda import FloatTensor, LongTensor

from keras.utils import to_categorical

import numpy as np

np.random.seed(42)

# Рекуррентные нейронные сети

## Simple RNN

Вообще говоря, я предполагаю, что вы уже несколько раз слышали про RNN.

Основная их прелесть - расшаренные параметры. Посмотрите на картинку:

![RNN types](http://karpathy.github.io/assets/rnn/diags.jpeg =x250)

[(from The Unreasonable Effectiveness of Recurrent Neural Networks)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) 

Первый пример - это обычная полносвязная сеть. Каждый следующий демонстрирует обработку некоторой последовательности произвольной длины (красные прямоугольнички) и генерацию выходной последовательности, также произвольной длины (синие прямоугольники).

При этом зеленые прямоугольники в каждом рисунке - это одни и те же веса. Получается, мы, с одной стороны, обучаем очень-очень глубокую сеть (если посмотреть на неё перевернутую), а с другой - строго ограниченное количество параметров.

---
Напишем сразу простую RNN!

Напомню, делает она примерно вот это:

![rnn-unrolled](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png =x220)

[(from Understanding LSTM Networks)](http://colah.github.io/posts/2015-08-Understanding-LSTMs)

Вообще говоря, можно придумать много вариаций на тему такой реализации. В нашем случае, обработка будет такой:
$$h_t = tanh(W_h [h_{t-1}; x_t] + b_h)$$

$h_{t-1}$ - скрытое состояние, полученное на предыдущем шаге, $x_t$ - входной вектор. $[h_{t-1}; x_t]$ - простая конкатенация векторов. Всё как на картинке!

Будем применять ещё и линейное отображение на пространство заданной выходной размерности для получения выходного вектора $s_t$:
$$s_t = W_o h_t + b_o$$

In [0]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()

        self.hidden_size = hidden_size
        
        self.hidden_linear = nn.Linear(input_size + hidden_size, hidden_size)
        self.output_linear = nn.Linear(hidden_size, output_size)

    def forward(self, inputs, hidden):
        combined = torch.cat((inputs, hidden), 1)
        
        hidden = self.hidden_linear(combined)
        hidden = F.tanh(hidden)
        
        output = self.output_linear(hidden)        
        return output, hidden

    def init_hidden(self, batch_size):
        return Variable(torch.zeros(batch_size, self.hidden_size).cuda())

Проверим нашу сеть на очень простой задаче: заставим её говорить индекс первого элемента в последовательности.

Т.е. для последовательности `[1, 2, 1, 3]` сеть должна предсказывать `1`.

Начнем с генерации батча. Батч содержит исключительно цифры, закодируем их с помощью one-hot-encoding представления.

In [0]:
def generate_data(batch_size=128, seq_len=5):
    data = np.random.randint(10, size=(seq_len, batch_size))

    X = Variable(FloatTensor(to_categorical(data)))
    y = Variable(LongTensor(data[0]))
    return X, y

X_val, y_val = generate_data()

Обратите внимание, что батч имеет размерность `(sequence_length, batch_size, input_size)`. Все `RNN` в pytorch работают с таким форматом по умолчанию. В keras эти размерности шли в другом порядке: сначала `batch_size`, потом `sequence_length`.

Сделано это из соображений производительности, но при желании можно поменять такое поведение с помощью аргумента `batch_first`.

In [0]:
rnn = SimpleRNN(input_size=10, hidden_size=32, output_size=10)
rnn = rnn.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters())

total_loss = 0
epochs_count = 1000
for epoch_ind in range(epochs_count):
    X_train, y_train = generate_data()
    
    optimizer.zero_grad()
    rnn.train()
    hidden = rnn.init_hidden(batch_size=128)
    for i in range(X_train.size()[0]):
        output, hidden = rnn(X_train[i], hidden)

    loss = criterion(output, y_train)
    loss.backward()
    
    nn.utils.clip_grad_norm(rnn.parameters(), 1.)
    optimizer.step()
    
    total_loss += loss.data[0]
    
    if epoch_ind != 0 and epoch_ind % 100 == 0:
        rnn.eval()
        hidden = rnn.init_hidden(batch_size=128)
        for i in range(X_val.size()[0]):
            output, hidden = rnn(X_val[i], hidden)
        val_loss = criterion(output, y_val)
        print('[{}/{}] Train: {:6f} Val: {:6f}'.format(epoch_ind, epochs_count, 
                                                       total_loss / 100, val_loss.data[0]))
        total_loss = 0

**Задание** Посмотрите на то, как влияет длина последовательности на работу сети. 

Во-первых, посмотри, с какой длиной сеть в состоянии учиться. Во-вторых, попробуйте обучить сеть с небольшой длиной последовательности, а потом применять её к более длинным.

*Почему получается то, что получается?*

**Задание** Утверждается, что `relu` подходит для RNN лучше. Попробуйте и её.

## LSTM и GRU



Если всё пошло по плану, мы должны были посмотреть на то, как RNN'ки забывают. 

Чтобы понять причину, стоит вспомнить, как именно происходит обучение RNN, например, здесь: [Backpropagation Through Time and Vanishing Gradients](http://www.wildml.com/2015/10/recurrent-neural-networks-tutorial-part-3-backpropagation-through-time-and-vanishing-gradients/) или здесь - [Vanishing Gradients & LSTMs](http://harinisuresh.com/2016/10/09/lstms/).

Если кратко, одна из проблем обучения рекуррентных сетей - *взрыв градиентов*. Она проявляется, когда матрица весов такова, что увеличивает норму вектора градиента при обратном проходе. В результате норма градиента экспоненциально растет и он "взрывается". 

Эту проблему мы решили с помощью клипинга градиентов: `nn.utils.clip_grad_norm(rnn.parameters(), 1.)`.

Другая проблема - *затухание градиентов*. Она связана наоборот - с экспоненциальным затуханием градиентов. И вот её решают уже более сложными способами. 

А именно - используют gate'овые архитектуры.

Идея gate'а простая, но важная, используются они далеко не только в рекуррентных сетях.

Если посмотреть на то, как работает наша SimpleRNN, можно заметить, что каждый раз память (т.е. $h_t$) перезаписывается. Хочется иметь возможность сделать эту перезапись контролируемой: не отбрасывать какую-то важную инфомацию из вектора.

Заведем для этого вектор $g \in {0,1}^n$, который будет говорить, какие ячейки $h_{t-1}$ хорошие, а вместо каких стоит подставить новые значения:
$$h_t = g \odot f(x_t, h_{t-1}) + (1 - g) \odot h_{t-1}.$$

Например:
$$
 \begin{bmatrix}
  8 \\
  11 \\
  3 \\
  7
 \end{bmatrix} =
 \begin{bmatrix}
  0 \\
  1 \\
  0 \\
  0
 \end{bmatrix}
 \odot
  \begin{bmatrix}
  7 \\
  11 \\
  6 \\
  5
 \end{bmatrix}
 +
  \begin{bmatrix}
  1 \\
  0 \\
  1 \\
  1
 \end{bmatrix}
 \odot
  \begin{bmatrix}
  8 \\
  5 \\
  3 \\
  7
 \end{bmatrix}
$$

Чтобы добиться дифференцируемости, будем использовать сигмоиду: $\sigma(f(x_t, h_{t-1}))$.

В результате сеть будет сама, глядя на входы, решать, какие ячейки своей памяти и насколько стоит перезаписывать.

### LSTM

Кажется, первой архитектурой, применившей данной механизм, стал LSTM (Long Short-Term Memory).

В ней у нас к $h_{t-1}$ добавляется ещё и $c_{t-1}$: $h_{t-1}$ - это всё то же скрытое состояния полученное на предыдущем шаге, а $c_{t-1}$ - это вектор памяти.

Для начала мы можем точно так же, как и раньше посчитать новое скрытое состояние (обозначим его $\tilde c_{t}$):
$$\tilde c_{t} = tanh(W_h [h_{t-1}; x_t] + b_h)$$

В обычных RNN мы бы просто перезаписали этим значением сторое скрытое состояние. А теперь мы хотим понять, насколько нам нужна информация из $c_{t-1}$ и из $\tilde c_{t}$. 

Оценим её сигмоидами:
$$f = \sigma(W_f [h_{t-1}; x_t] + b_f),$$
$$i = \sigma(W_i [h_{t-1}; x_t] + b_i).$$

Первая - про то, насколько хочется забыть старую информацию. Вторая - насколько интересна новая. Тогда
$$c_t = f \odot c_{t-1} + i \odot \tilde c_t.$$

Новое скрытое состояние мы также взвесим:
$$o = \sigma(W_o [h_{t-1}; x_t] + b_o),$$
$$h_t = o \odot tanh(c_t).$$

Настоятельно рекомендуется почитать статью: [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) для более подробного ознакомления и прикольных картинок.

Зачем я выписал эти формулы? Главное - чтобы показать, насколько больше параметров нужно учить в LSTM по сравнению с обычным RNN. В четыре раза больше!

## Классификация имён

Начнём уже решать задачи. 

Сперва попробуем классифицировать имена: потренируем сеть символьного уровня, которая будет говорить, из какого языка пришло такое имя.

Должно получаться что-то вроде:

```
Hinton
(-0.47) Scottish
(-1.52) English
(-3.57) Irish

Schmidhuber
(-0.19) German
(-2.48) Czech
(-2.68) Dutch
```

Скачаем данные.

In [0]:
!wget -O data.zip https://download.pytorch.org/tutorial/data.zip
!unzip data.zip

In [0]:
import unicodedata
import string
import os

all_letters = string.ascii_letters + ".,;'"
n_letters = len(all_letters)

# 0 - индекс паддинга!
char2index = {c : i + 1 for i, c in enumerate(all_letters)}
char2index['</s>'] = max(char2index.values()) + 1
char2index['<unk>'] = max(char2index.values()) + 1
index2char = {char2index[c] : c for c in char2index}

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    with open(filename, encoding='utf-8') as f:
        return [unicodeToAscii(line.strip()) for line in f]

files = ['data/names/' + file for file in os.listdir('data/names') if file.endswith('.txt')]  

for filename in files:
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

Посмотрим на результат:

In [0]:
[(x, category_lines[x][:3]) for x in category_lines]

Сконвертируем датасет:

In [0]:
words_count = sum(len(category_lines[x]) for x in category_lines)
max_word_len = max(max(len(w) for w in category_lines[x]) for x in category_lines)

X = np.zeros((words_count, max_word_len))
y = np.zeros(words_count)
i = 0
for category_ind, category in enumerate(category_lines):
    for word in category_lines[category]:
        X[i, :len(word)] = [char2index[symb] for symb in word]
        y[i] = category_ind
        i += 1
        
indices = np.random.permutation(np.arange(X.shape[0]))

X = X[indices]
y = y[indices]

X_train, y_train = X[: 3 * X.shape[0] // 4], y[: 3 * X.shape[0] // 4]
X_val, y_val = X[3 * X.shape[0] // 4 :], y[3 * X.shape[0] // 4 :]

X_train, X_val = LongTensor(X_train), LongTensor(X_val)
y_train, y_val = LongTensor(y_train), LongTensor(y_val)

А теперь задумаемся. Какая нам нужна размерность, чтобы подать тензор на вход LSTM?

In [0]:
def get_batches(dataset, batch_size):
    X, y = dataset
    n_samples = X.shape[0]

    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        batch_idx = indices[start:end]
        
        X_batch = X[batch_idx, ].transpose(1, 0)
        y_batch = y[batch_idx, ]
        
        yield Variable(X_batch), Variable(y_batch)

Обратите внимание на `transpose`!

Построим LSTM модель.

In [0]:
class LstmClassifier(nn.Module):
    def __init__(self, char_vocab_size, char_embedding_dim, 
                 lstm_hidden_dim, classes_count, batch_size):
        super().__init__()
        
        self.lstm_hidden_dim = lstm_hidden_dim
        self.char_embedding = nn.Embedding(char_vocab_size, char_embedding_dim)
        self.lstm = nn.LSTM(char_embedding_dim, lstm_hidden_dim)
        self.output = nn.Linear(lstm_hidden_dim, classes_count)
        
        self.hidden = self.init_hidden(batch_size)
        
    def forward(self, word):
        chars = self.char_embedding(word)
        
        _, self.hidden = self.lstm(chars, self.hidden)
        
        pred = self.output(self.hidden[0])
        return pred.view(pred.size()[1:])
    
    def init_hidden(self, batch_size):
        return (Variable(torch.zeros(1, batch_size, self.lstm_hidden_dim).cuda()),
                Variable(torch.zeros(1, batch_size, self.lstm_hidden_dim).cuda()))

In [0]:
BATCH_SIZE = 32
model = LstmClassifier(len(char2index) + 1, 32, 64, n_categories, BATCH_SIZE)
model = model.cuda()

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

total_loss = 0
for _ in range(100):
    for i, (X_batch, y_batch) in enumerate(get_batches((X_train, y_train), BATCH_SIZE)):
        optimizer.zero_grad()

        model.hidden = model.init_hidden(y_batch.size()[0])

        preds = model(X_batch)

        loss = loss_function(preds, y_batch)
        loss.backward()
        total_loss += loss.data[0]

        optimizer.step()

        if i != 0 and i % 100 == 0:
            print(total_loss / 100)
            total_loss = 0
            
            <evaluate model on val set, please>

**Задание** Напишите функцию для тестирования полученной сети: пусть она принимает слово и говорит, в каком языке с какой вероятностью это может быть фамилией.

**Задание** Важным видом RNN является Bidirectional RNN. По сути это две RNN, одна обходит последовательность слева направо, вторая - наоборот. 

В результате для каждого момента времени у нас есть вектор $h_t = [f_t; b_t]$ - конкатенация (или какая-то ещё функция от $f_t$ и $b_t$) состояний $f_t$ и $b_t$ - прямого и обратного прохода последовательности. В сумме они покрывают весь контекст.

В нашей задаче Bidirectional вариант может помочь тем, что сеть будет меньше забывать, с чего начиналась последовательность. То есть нам нужно будет взять $f_N$ и $b_N$ состояния: первое - последнее состояние в проходе слева направо, т.е. выход от последнего символа. Второе - последнее состояние при обратно проходе, т.е. выход для первого символа.

Реализуйте Bidirectional классификатор. Для этого в `LSTM` есть параметр `bidirectional`.


**Задание** Сейчас память расходуется очень неоптимально: мы создали батчи длиной равно длине максимального слова. Лучше разбить все слова по группам, например: слова с длиной до 4, до 8, до 16 и больше 16. 

При этом стоит варьировать и размер батча. Например, 32 - для 16, 64 - для 8 и т.д. То есть число символов в батче должно быть константным.

Реализуйте такой вариант обучения.

## Генерация имён

Просто классифицировать - не интересно. Давайте попробуем генерировать имена из нужного языка!

Хотим получить что-нибудь такое:

![generation](https://i.imgur.com/JH58tXY.png)

То есть для каждого символа мы будем учиться предсказывать следующий - до тех пор, пока не сгенерируем символ конца последовательности `EOS`.

Начнём, как всегда, с построения батчей.

In [0]:
X = np.zeros((words_count, max_word_len))
y = np.zeros((words_count, max_word_len))
X_category = np.zeros((words_count, n_categories))
i = 0
for category_ind, category in enumerate(category_lines):
    for word in category_lines[category]:
        X[i, :len(word)] = [char2index[symb] for symb in word]
        y[i, :len(word)] = [char2index[symb] for symb in word[1:]] + [char2index['</s>']]
        X_category[i, category_ind] = 1
        i += 1
        
indices = np.random.permutation(np.arange(X.shape[0]))

X = X[indices]
y = y[indices]
X_category = X_category[indices]

train_size = 3 * X.shape[0] // 4
X_train, y_train, X_category_train = X[: train_size].T, y[: train_size].T, X_category[: train_size]
X_val, y_val, X_category_val = X[train_size :].T, y[train_size :].T, X_category[train_size :]

X_train, X_val = LongTensor(X_train), LongTensor(X_val)
y_train, y_val = LongTensor(y_train), LongTensor(y_val)
X_category_train, X_category_val = FloatTensor(X_category_train), FloatTensor(X_category_val)

In [0]:
def get_batches(dataset, batch_size):
    X, y, category = dataset
    n_samples = X.shape[1]

    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        batch_idx = indices[start:end]
        
        X_batch = X[:, batch_idx]
        y_batch = y[:, batch_idx]
        category_batch = category[batch_idx, ].expand(X_batch.size()[:1] + category[batch_idx, ].size())
        
        yield Variable(X_batch), Variable(y_batch.view(-1)), Variable(category_batch)

Обратите внимание: категория - one-hot-encoding вектор. Вообще говоря, можно было бы обучать эмбеддинги для категорий (*кстати, попробуйте*), но кажется логичным ожидать, что вектора для разных языков ортогональны друг другу.

Напишем код для класса-генератора.

Теперь нам нужен уже не выход $h_n$ из пары $(h_n, c_n)$ - финальное состояние, которым мы пользовались раньше, а весь $\text{output} = (h_1, \ldots, h_n)$ - состояния `lstm` для всех моментов времени. Это первый элемент, возвращаемый `lstm`. 

По ним мы и будем предсказывать символ: символ в позиции $t$ должен получаться сэмлированием из вектора, заданного линейным преобразованием состояния `lstm` $h_t$: $\ \text{symb}_t \sim W_o h_t + b_o$. (Это линейное преобразование задает `self.output`).

Подумайте, какие размерности будет иметь выход из `lstm` и результат его обработки с помощью полносвязного слоя.

In [0]:
class LstmGenerator(nn.Module):
    def __init__(self, char_vocab_size, char_embedding_dim, category_count,
                 lstm_hidden_dim, classes_count, batch_size):
        super().__init__()
        
        self.lstm_hidden_dim = lstm_hidden_dim
        self.char_embedding = nn.Embedding(char_vocab_size, char_embedding_dim)
        self.lstm = nn.LSTM(char_embedding_dim + category_count, lstm_hidden_dim)
        self.output = nn.Linear(lstm_hidden_dim, classes_count)
        
        self.hidden = self.init_hidden(batch_size)
        
    def forward(self, word, category):
        <your code here>
    
    def init_hidden(self, batch_size):
        return (Variable(torch.zeros(1, batch_size, self.lstm_hidden_dim).cuda()),
                Variable(torch.zeros(1, batch_size, self.lstm_hidden_dim).cuda()))

Потренируем модель. Делается это аналогично предыдущей модели.

In [0]:
BATCH_SIZE = 32
model = LstmGenerator(len(char2index) + 1, 32, n_categories, 64, len(char2index) + 1, BATCH_SIZE)
model = model.cuda()

<your code here>

А теперь попробуем погенерировать какой-нибудь текст. Пригодится такая функция:

In [0]:
def sample(preds, temperature=1.0):
    preds = preds[0, 0] / temperature
    preds = F.softmax(preds, dim=0).data.cpu().numpy()
    return np.random.choice(np.arange(preds.shape[0]), p=preds)

**Задание** Написать функцию-генератор фамилии, принимающую первую букву и язык, из которого должна быть эта фамилия.

Она должна в цикле генерировать слово символ за символом, пока не выпадет `</s>`. Для генерации символа нужна функция `sample`. Обратите внимание на параметр `temperature`. Чем он ниже, тем консервативнее модель (т.е. ниже вероятность предсказать маловероятный символ). Чем выше - тем разнообразнее её предсказания.

Обратите внимание: нам не нужно подавать в сеть весь предыдущий предсказанный контекст. Например, если модель только что предсказала $p_{t-1}$ - распределение вероятностей для $t-1$ позиции, она также помнит и $(h_{t-1}, c_{t-1})$ - вектора, содержащие состояние модели. Они записаны в `self.hidden`.

Тогда достаточно передать модели $c_{t-1}$ - сэмплированный из $p_{t-1}$ элемент, чтобы она смогла предсказать $p_{t}$. То есть передавать в модель будем на каждом шаге тензор с одним элементом, а обнулять `self.hidden` - только перед началом генерации.

In [0]:
def generate_surname(model, category=1, first_letter='A', temperature=0.5):
    <your code here>

## Безусловная генерация текстов

Вообще говоря, то, что вы видели выше - условная генерация текста. Мы там генерировали фамилии при условии заданного языка. Есть и более простая разновидность генерации текстов - безусловная.

Попробуем погенерировать Шекспира!

In [0]:
!wget --quiet -c -O shakespeare.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

Файл выглядит так:

In [6]:
!head shakespeare.txt

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:


**Задание** Постройте модель, которая будет генерировать текст, обучившись на Шекспире.

Это должна быть символьная модель - совсем как те две, которые мы делали перед этим.

В качестве батча нужно будет подавать тензоры размерности `(sequence_length, batch_size)`.

Скорее всего, правильнее будет превратить последовательность символов, записанную в `shakespeare.txt` в вектор индексов этих символов длины `N`. 

Дальше - превратить его в матрицу `(N // BS, BS)`, где `BS` - выбранный размер батча. Для этого подойдет простой вызов `view`. Только нужно перед ним либо обрезать `N` до значения, делящегося нацело на `BS`, либо наоборот - дополнить нулями вектор индексов.

Например, если изначальная последовательность была просто последовательностью всех букв от `a` до `z`, матрица может быть следующей при размере батча 4:
```
┌ a g m s ┐
│ b h n t │
│ c i o u │
│ d j p v │
│ e k q w │
└ f l r x ┘
```

Затем нам нужно будет формировать мини-батч просто двигаясь по этой матрице. 

При длине последовательности 2 наши `X` и `y` будут такими:
```
    ┌ a g m s ┐       ┌ b h n t ┐
X = └ b h n t ┘,  y = └ c i o u ┘
```

При этом hidden state мы можем сохранять - в начале нового мини-батча моделе будет полезно знать, чем закончился предыдущий. Для этого пригодится такая функция:

In [0]:
def repackage_hidden(h):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

Have fun! :)